In [1]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

df = pd.read_csv('../allRunsAllFractions.csv')

# In einem ersten Ansatz wurden alle Fraktion als Zahlwert mit einbezogen
conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #
]

values = [1, 2, 3]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "fraction", "time"])
len(dfPrepSample)
dfPrepSample


,total_workers,resource_mining,total_army_value,total_army,supply
1,46,28154,5825,96.0,174.0
3,72,39176,9750,126.0,200.0
4,35,22633,800,14.0,70.0
6,66,22585,2375,40.0,117.0
12,77,35585,6800,106.0,181.0
...,...,...,...,...,...
12654,80,49553,8525,110.0,200.0
12655,73,49464,8550,123.0,200.0
12659,70,26351,4625,70.0,157.0
12661,70,24263,5300,76.0,165.0


In [2]:
# optimizer with scaling and lin reg

import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr


dfOpt = dfPrepSample.drop(columns=[
                               "total_workers"])




In [3]:


# create variables
# create Model

m = gp.Model("Balance optimizer")
gppd.set_interactive()

av = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army_value")

a = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army")
s = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="supply")
w = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_workers", vtype=gp.GRB.INTEGER)

m.update()
dfOpt = dfOpt.drop(columns=["total_army_value"])

feats = dfOpt
feats = pd.concat([w, feats], axis=1)
feats


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


,total_workers,resource_mining,total_army,supply
1,<gurobi.Var total_workers[1]>,28154,96.0,174.0
3,<gurobi.Var total_workers[3]>,39176,126.0,200.0
4,<gurobi.Var total_workers[4]>,22633,14.0,70.0
6,<gurobi.Var total_workers[6]>,22585,40.0,117.0
12,<gurobi.Var total_workers[12]>,35585,106.0,181.0
...,...,...,...,...
12654,<gurobi.Var total_workers[12654]>,49553,110.0,200.0
12655,<gurobi.Var total_workers[12655]>,49464,123.0,200.0
12659,<gurobi.Var total_workers[12659]>,26351,70.0,157.0
12661,<gurobi.Var total_workers[12661]>,24263,76.0,165.0


In [4]:
sc_model = tf.keras.models.load_model('trained_model.h5')


In [5]:
# ein Versuch die Zeit zu verkürzen, entnommen von: https://support.gurobi.com/hc/en-us/community/posts/8315312763409-Limit-solutions-for-an-LP
def simplexcallback(model,where):
  if where == gp.GRB.Callback.SIMPLEX:
      vio = model.cbGet(gp.GRB.Callback.SPX_PRIMINF)
      if vio < 1e-6:
        print(model.cbGet(gp.GRB.Callback.SPX_OBJVAL))
        print("Reached primal feasibility")
        model.terminate()

m.setObjective(av.sum(), gp.GRB.MAXIMIZE) # Zielfunktion

gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Untergrenze Versorgung
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, s) # Obergrenze Versorgung
m.update()

pred_constr = add_predictor_constr(m, sc_model, feats, av) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.Params.NonConvex = 2
m.optimize(simplexcallback)


Model for sequential1:
581584 variables
289008 constraints
285440 general constraints
Input has shape (3568, 4)
Output has shape (3568, 1)

--------------------------------------------------------------------------------
Layer           Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
dense0            (3568, 50)       356800       178400            0       178400 (relu)

dense1            (3568, 30)       214080       107040            0       107040 (relu)

dense2             (3568, 1)            0         3568            0            0

--------------------------------------------------------------------------------
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimi

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\gurobipy\callback.pxi", line 209, in gurobipy.CallbackClass.callback
  File "C:\Users\nicol\AppData\Local\Temp\ipykernel_1948\204377610.py", line 1, in simplexcallback
KeyboardInterrupt: 


In [ ]:


outputs = pred_constr.input_values
pred_constr.input_values



In [ ]:
# Ausgeben verschiedener Werte als Dataframe (Übernommen aus https://gurobi-machinelearning.readthedocs.io/en/stable/mlm-examples/price_optimization.html#Fire-Up-the-Solver)
solution = pd.DataFrame()

solution["ratio"] = r.gppd.X
solution["supply"] = s.gppd.X
solution["army"] = a.gppd.X
solution["worker"] = w
solution


In [ ]:

custom_palette = ["#dd8452","#4c72b0",  "#55a868"]
sns.set_theme(style="ticks")
# Show the joint distribution using kernel density estimation
g = sns.jointplot(
    data=solution, palette=custom_palette,
    x="ratio", y="supply", 
    kind="kde", height=10
)